# Traffic Application Testing Code

This notebook is designed to communicate with the Traffic Application server and perform a series of tests. It is necessary to restart the server for each test to ensure that the test did not fail because of the state the server was in due to the previous tests. Each time you restart the server ensure that it begins with a database in a known state.

In [1]:
# The modules needed
import random
import requests
import shutil
import json
import subprocess
import csv

In [2]:
# The port the server should run on

server_port = '8081'

In [3]:
# The core code

def request2server(url,cookies):
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.get(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]),timeout=30)
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        act=json.loads(r.text)
        return [[ucookie,mcookie],act]
    except :
        print("Invalid login")
        return [[ucookie,mcookie],[]]
    
def requestfilefromserver(url,cookies):
    ucookie = cookies[0]
    mcookie = cookies[1]
    try:
        r = requests.get(url,cookies=dict(u_cookie=cookies[0],m_cookie=cookies[1]),timeout=15)
        for c in r.cookies:
            if (c.name == 'u_cookie'):
                ucookie = c.value
            if (c.name == 'm_cookie'):
                mcookie = c.value
        return [[ucookie,mcookie],r.text]
    except :
        print("Invalid login")
        return [[ucookie,mcookie],[]]
    
def find_redirect(act):
    for a in act:
        if 'type' in a:
            if a['type'] == 'redirect':
                if 'where' in a:
                    return a['where']
    return None

def find_refill(act,where):
    for a in act:
        if 'type' in a:
            if a['type'] == 'refill':
                if 'where' in a:
                    if a['where'] == where:
                        return a['what']
    return None

def randparam():
    return "&randn="+str(random.randrange(10000,99999))

def do_login(cookies,user,pasw):
    return request2server("http://localhost:"+server_port+"/action?command=login&usernameinput="+user+"&passwordinput="+pasw+randparam(),cookies)

def do_add(cookies,loc,occupancy,vtype):
    return request2server("http://localhost:"+server_port+"/action?command=add&locationinput="+loc+"&occupancyinput="+str(occupancy)+"&typeinput="+vtype+randparam(),cookies)

def do_undo(cookies,loc,occupancy,vtype):
    return request2server("http://localhost:"+server_port+"/action?command=undo&locationinput="+loc+"&occupancyinput="+str(occupancy)+"&typeinput="+vtype+randparam(),cookies)

def do_summary(cookies):
    return request2server("http://localhost:"+server_port+"/action?command=summary"+randparam(),cookies)

def do_logout(cookies):
    return request2server("http://localhost:"+server_port+"/action?command=logout"+randparam(),cookies)

def do_file_fetch(cookies,file):
    return requestfilefromserver("http://localhost:"+server_port+"/"+file,cookies)

In [4]:
def checked_login(test,cookies,user,pasw):
    [cookies,act] = do_login(cookies, user,pasw)
    where = find_redirect(act)
    if(where == None):
        print("Test "+str(test)+" Failed - Expected redirect during login.")
        return ['',act,False]
    if (where != '/page.html'):
        print("Test "+str(test)+" Failed - Expected /page.html got {"+where+"}")
        return [cookies,act,False]
    return [cookies,act,True]

def checked_logout(test,cookies):
    [cookies,act] = do_logout(cookies)     
    where = find_redirect(act)
    
    if (where != '/index.html'):
        print("Test "+str(test)+" Failed - Expected /index.html got {"+where+"}")
        return [cookies,act,False] 
    return [cookies,act,True]

In [5]:
# Test 1 - Simple login
def test1():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(1,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0

        [cookies,act,flag] = checked_logout(1,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 1 Passed")
        return 1
    except:
        print("Test 1 Failed - Exception Caused.")
        return 0


In [6]:
# Test 2 - Bad login
def test2():
    try:
        cookies = ['','']
        [cookies,act2] = do_login(cookies,"test1","passwor1")
        where = find_redirect(act2)
    
        if (where != None):
            print("Test 2 Failed - Unexpected redirect.")
            return 0
        
        message = find_refill(act2,'message')
    
        if (message == None):
            print("Test 2 Failed - No failure message present")
            return 0
        
        [cookies,act,flag] = checked_logout(2,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 2 Passed")
        return 1
    except:
        print("Test 2 Failed - Exception Caused.")
        return 0        

In [7]:
# Test 3 - Injection attack
def test3():
    try:
        cookies = ['','']
        [cookies,act] = do_login(cookies,"test1",'"'+' OR "0"="0')
        
        where = find_redirect(act)
        if(where == None):
            message = find_refill(act,'message')
    
            if (message == None):
                print("Test 3 Failed - No failure message present when not re-directing.")
                return 0
                
        elif (where != '/index.html'):
            print("Test 3 Failed - Expected /index.html got {"+where+"}")
            return 0
        
        [cookies,act,flag] = checked_logout(3,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 3 Passed")
        return 1
    except:
        print("Test 3 Failed - Exception Caused.")
        return 0

In [8]:
# Test 4 - Simple add
def test4():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(4,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 4 Failed - Unexpected redirect.(2)")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 4 Failed - No acknowledge message present")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 4 Failed - No total message present")
            return 0
        
        if (total != "1"):
            print("Test 4 Failed - Total not expected value")
            return 0
         
        [cookies,act,flag] = checked_logout(4,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 4 Passed")
        return 1
    except:
        print("Test 4 Failed - Exception Caused.")
        return 0

In [9]:
# Test 5 - Looping Add
def test5():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(5,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        i = 1
        
        while i < 5:
            
          [cookies_x,act] = do_add(cookies,"road",1,"car")
      
          where = find_redirect(act)
    
          if (where != None):
            print("Test 5 Failed - Unexpected redirect.(2)")
            return 0
        
          message = find_refill(act,'message')
    
          if (message == None):
            print("Test 5 Failed - No acknowledge message present")
            return 0
        
          total = find_refill(act,'total')
    
          if (total == None):
            print("Test 5 Failed - No total message present")
            return 0
        
          if (total != str(i)):
            print("Test 5 Failed - Total not expected value {"+str(i)+"}")
            return 0
        
          i = i + 1
        
        [cookies,act,flag] = checked_logout(5,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 5 Passed")
        return 1
    except:
        print("Test 5 Failed - Exception Caused.")
        return 0

In [10]:
# Test 6 - Per session summary
def test6():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(6,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        i = 1
        
        while i < 5:
            
          [cookies_x,act] = do_add(cookies,"road",1,"car")
      
          where = find_redirect(act)
    
          if (where != None):
            print("Test 6 Failed - Unexpected redirect.(2)")
            return 0
        
          message = find_refill(act,'message')
    
          if (message == None):
            print("Test 6 Failed - No acknowledge message present")
            return 0
        
          total = find_refill(act,'total')
    
          if (total == None):
            print("Test 6 Failed - No total message present")
            return 0
        
          if (total != str(i)):
            print("Test 6 Failed - Total not expected value {"+str(i)+"} Initial Step")
            return 0
        
          i = i + 1
        
        [cookies,act,flag] = checked_logout(6,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        cookies = ['','']
        [cookies,act,flag] = checked_login(6,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        i = 1
        
        while i < 5:
            
          [cookies_x,act] = do_add(cookies,"road",1,"car")
      
          where = find_redirect(act)
    
          if (where != None):
            print("Test 6 Failed - Unexpected redirect.(2)")
            return 0
        
          message = find_refill(act,'message')
    
          if (message == None):
            print("Test 6 Failed - No acknowledge message present")
            return 0
        
          total = find_refill(act,'total')
    
          if (total == None):
            print("Test 6 Failed - No total message present")
            return 0
        
          if (total != str(i)):
            print("Test 6 Failed - Total not expected value {"+str(i)+"} Second Step")
            return 0
        
          i = i + 1
        
        [cookies,act,flag] = checked_logout(6,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 6 Passed")
        return 1
    except:
        print("Test 6 Failed - Exception Caused.")
        return 0

In [11]:
# Test 7 - Per session summary with mixed traffic
def test7():
    types=["car","bicycle","bus","other","car","van","motorbike"]
    length = len(types)
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(7,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        i = 1
        
        for type in types:
            
          [cookies_x,act] = do_add(cookies,"road",1,type)
      
          where = find_redirect(act)
    
          if (where != None):
            print("Test 7 Failed - Unexpected redirect.(2)")
            return 0
        
          message = find_refill(act,'message')
    
          if (message == None):
            print("Test 7 Failed - No acknowledge message present")
            return 0
        
          total = find_refill(act,'total')
    
          if (total == None):
            print("Test 7 Failed - No total message present")
            return 0
        
          if (total != str(i)):
            print("Test 7 Failed - Total not expected value {"+str(i)+"} Initial Step")
            return 0
        
          i = i + 1
        
        [cookies,act,flag] = checked_logout(7,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 7 Passed")
        return 1
    except:
        print("Test 7 Failed - Exception Caused.")
        return 0

In [12]:
# Test 8 - Unknown vehicle type, step 1
def test8():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(8,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
    
        [cookies_x,act] = do_add(cookies,"road",1,"caravan")
        where = find_redirect(act)
    
        if (where != None):
            print("Test 8 Failed - Unexpected redirect.(2)")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 8 Failed - No acknowledge message present(3)")
            return 0
        
        total = find_refill(act,'total')
    
        if (total != None):
            if (total != "0"):
                print("Test 8 Failed - Total not expected value, should be zero.")
                return 0
        
        [cookies,act,flag] = checked_logout(8,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 8 Passed")
        return 1
    except:
        print("Test 8 Failed - Exception Caused.")
        return 0

In [13]:
# Test 9 - Unknown vehicle type, step 2
def test9():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(9,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0    
            
        [cookies_x,act] = do_add(cookies,"road",1,"caravan")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 9 Failed - Unexpected redirect.(2)")
            return 0
        
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 9 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
    
        if (total != None):
            if (total != "1"):
                print("Test 9 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies,act,flag] = checked_logout(9,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 9 Passed")
        return 1
    except:
        print("Test 9 Failed - Exception Caused.")
        return 0

In [14]:
# Test 10 - Simple undo
def test10():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(10,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
    
            
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 10 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 10 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies_x,act] = do_undo(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 10 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total == None):
            print("Test 10 Failed - Total not found.")
            return 0
    
        if (total != "0"):
            print("Test 10 Failed - Total not expected value, should be 0. Was {"+total+"}")
            return 0
        
        [cookies,act,flag] = checked_logout(10,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 10 Passed")
        return 1
    except:
        print("Test 10 Failed - Exception Caused.")
        return 0

In [15]:
# Test 11 - More cars than seen
def test11():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(11,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
    
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 11 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 11 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies_x,act] = do_undo(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 11 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total == None):
            print("Test 11 Failed - Total not found.")
            return 0
    
        if (total != "0"):
            print("Test 11 Failed - Total not expected value, should be 0 (A). Was {"+total+"}")
            return 0
        
        [cookies_x,act] = do_undo(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 11 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total == None):
            print("Test 11 Failed - Total not found.")
            return 0
    
        if (total != "0"):
            print("Test 11 Failed - Total not expected value, should be 0 (B). Was {"+total+"}")
            return 0
        
        [cookies,act,flag] = checked_logout(11,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 11 Passed")
        return 1
    except:
        print("Test 11 Failed - Exception Caused.")
        return 0

In [16]:
# Test 12 - Undo wrong kind of vehicle
def test12():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(12,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
            
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 12 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 12 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies_x,act] = do_undo(cookies,"road",1,"bus")
      
        
        message = find_refill(act,'message')
        
        if (message == None):
            print("Test 12 Failed - No error message present")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):   
            if (total != "1"):
                print("Test 12 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
            
        where = find_redirect(act)
        
        if (where != None):
            print("Test 12 Failed - Unexpected redirect.(2)")
            return 0
        
        [cookies,act,flag] = checked_logout(12,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 12 Passed")
        return 1
    except:
        print("Test 12 Failed - Exception Caused.")
        return 0

In [17]:
# Test 13 - Everyone's a winner.
def test13():
    print("Test 13 Passed")
    return 1

In [18]:
# Test 14 - Add out of range occupancy
def test14():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(14,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0    
            
        [cookies_x,act] = do_add(cookies,"road",5,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 14 Failed - Unexpected redirect.(2)")
            return 0

        message = find_refill(act,'message')
        
        if (message == None):
            print("Test 14 Failed - No error message present")
            return 0

        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "0"):
                print("Test 14 Failed - Total not expected value, should be 0. Was {"+total+"}")
                return 0
        
        [cookies,act,flag] = checked_logout(14,cookies)        
        where = find_redirect(act)
        
        if flag!=True:
            return 0

        print("Test 14 Passed")
        return 1
    except:
        print("Test 14 Failed - Exception Caused.")
        return 0

In [19]:
# Test 15 - Undo occupancy out of range
def test15():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(15,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
            
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 15 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 15 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies_x,act] = do_undo(cookies,"road",5,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 15 Failed - Unexpected redirect.(2)")
            return 0
    
        message = find_refill(act,'message')
        
        if (message == None):
            print("Test 15 Failed - No error message present")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):   
            if (total != "1"):
                print("Test 15 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies,act,flag] = checked_logout(15,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 15 Passed")
        return 1
    except:
        print("Test 15 Failed - Exception Caused.")
        return 0

In [20]:
# Test 16 - Injection into undo location
def test16():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(16,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
             
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 16 Failed - Unexpected redirect.(2)")
            return 0
        
        message = find_refill(act,'message')
        
        if (message == None):
            print("Test 15 Failed - No error message present")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 16 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies_x,act] = do_undo(cookies,'"OR ""=""',1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 16 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 16 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies,act,flag] = checked_logout(16,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 16 Passed")
        return 1
    except:
        print("Test 16 Failed - Exception Caused.")
        return 0

In [21]:
# Test 17 - Injection into undo type
def test17():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(17,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
             
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 17 Failed - Unexpected redirect.(2)")
            return 0
        
        message = find_refill(act,'message')
        
        if (message == None):
            print("Test 17 Failed - No error message present")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 17 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0

        [cookies_x,act] = do_undo(cookies,"road",'1','" OR ""="')
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 17 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 17 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
            
        message = find_refill(act,'message')
        if (message == None):
                print("Test 17 Failed - Message not found")
                return 0
        
        [cookies,act,flag] = checked_logout(16,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 17 Passed")
        return 1
    except:
        print("Test 17 Failed - Exception Caused.")
        return 0

In [22]:
# Test 18 - summary present
def test18():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(18,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
             
        [cookies_x,act] = do_summary(cookies)
        
        sum_car = find_refill(act,'sum_car')
        if (sum_car == None):
            print("Test 18 Failed - sum_car not present")
            return 0
            
        if (sum_car != "0"):
            print("Test 18 Failed - sum_car not expected value, should be 0. Was {"+total+"}")
            return 0
        
        sum_taxi = find_refill(act,'sum_taxi')
        if (sum_taxi == None):
            print("Test 18 Failed - sum_taxi not present")
            return 0
            
        if (sum_taxi != "0"):
            print("Test 18 Failed - sum_taxi not expected value, should be 0. Was {"+total+"}")
            return 0
        
        sum_bicycle = find_refill(act,'sum_bicycle')
        if (sum_bicycle == None):
            print("Test 18 Failed - sum_bicycle not present")
            return 0
            
        if (sum_bicycle != "0"):
            print("Test 18 Failed - sum_bicycle not expected value, should be 0. Was {"+total+"}")
            return 0
        
        sum_van = find_refill(act,'sum_van')
        if (sum_van == None):
            print("Test 18 Failed - sum_van not present")
            return 0
            
        if (sum_van != "0"):
            print("Test 18 Failed - sum_van not expected value, should be 0. Was {"+total+"}")
            return 0
        
        sum_truck = find_refill(act,'sum_truck')
        if (sum_truck == None):
            print("Test 18 Failed - sum_truck not present")
            return 0
            
        if (sum_truck != "0"):
            print("Test 18 Failed - sum_truck not expected value, should be 0. Was {"+total+"}")
            return 0
        
        sum_motorbike = find_refill(act,'sum_motorbike')
        if (sum_motorbike == None):
            print("Test 18 Failed - sum_motorbike not present")
            return 0
            
        if (sum_motorbike != "0"):
            print("Test 18 Failed - sum_motorbike not expected value, should be 0. Was {"+total+"}")
            return 0
        
        sum_bus = find_refill(act,'sum_bus')
        if (sum_car == None):
            print("Test 18 Failed - sum_bus not present")
            return 0
            
        if (sum_bus != "0"):
            print("Test 18 Failed - sum_bus not expected value, should be 0. Was {"+total+"}")
            return 0
        
        sum_other = find_refill(act,'sum_other')
        if (sum_other == None):
            print("Test 18 Failed - sum_other not present")
            return 0
            
        if (sum_other != "0"):
            print("Test 18 Failed - sum_other not expected value, should be 0. Was {"+total+"}")
            return 0
        
        [cookies,act,flag] = checked_logout(18,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 18 Passed")
        return 1
    except:
        print("Test 18 Failed - Exception Caused.")
        return 0

In [23]:
# Test 19 - summary present with some added traffic
def test19():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(19,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        # ignoring any errors here
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",2,"car")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle")
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",2,"motorbike")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle") 
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"taxi")
        [cookies,act] = do_add(cookies,"nowhere",3,"taxi")
        [cookies,act] = do_add(cookies,"road road",2,"van")
        [cookies,act] = do_add(cookies,"nowhere",4,"truck")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",3,"other")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle")
        [cookies,act] = do_summary(cookies)
        
        sum_car = find_refill(act,'sum_car')
        if (sum_car == None):
            print("Test 19 Failed - sum_car not present")
            return 0
            
        if (sum_car != "4"):
            print("Test 19 Failed - sum_car not expected value, should be 4. Was {"+sum_car+"}")
            return 0
        
        sum_taxi = find_refill(act,'sum_taxi')
        if (sum_taxi == None):
            print("Test 19 Failed - sum_taxi not present")
            return 0
            
        if (sum_taxi != "2"):
            print("Test 19 Failed - sum_taxi not expected value, should be 2. Was {"+sum_taxi+"}")
            return 0
        
        sum_bicycle = find_refill(act,'sum_bicycle')
        if (sum_bicycle == None):
            print("Test 19 Failed - sum_bicycle not present")
            return 0
            
        if (sum_bicycle != "3"):
            print("Test 19 Failed - sum_bicycle not expected value, should be 3. Was {"+sum_bicycle+"}")
            return 0
        
        sum_van = find_refill(act,'sum_van')
        if (sum_van == None):
            print("Test 19 Failed - sum_van not present")
            return 0
            
        if (sum_van != "1"):
            print("Test 19 Failed - sum_van not expected value, should be 1. Was {"+sum_van+"}")
            return 0
        
        sum_truck = find_refill(act,'sum_truck')
        if (sum_truck == None):
            print("Test 19 Failed - sum_truck not present")
            return 0
            
        if (sum_truck != "1"):
            print("Test 19 Failed - sum_truck not expected value, should be 1. Was {"+sum_truck+"}")
            return 0
        
        sum_motorbike = find_refill(act,'sum_motorbike')
        if (sum_motorbike == None):
            print("Test 19 Failed - sum_motorbike not present")
            return 0
            
        if (sum_motorbike != "1"):
            print("Test 19 Failed - sum_motorbike not expected value, should be 1. Was {"+sum_motorbike+"}")
            return 0
        
        sum_bus = find_refill(act,'sum_bus')
        if (sum_car == None):
            print("Test 19 Failed - sum_bus not present")
            return 0
            
        if (sum_bus != "3"):
            print("Test 19 Failed - sum_bus not expected value, should be 3. Was {"+sum_bus+"}")
            return 0
        
        sum_other = find_refill(act,'sum_other')
        if (sum_other == None):
            print("Test 19 Failed - sum_other not present")
            return 0
            
        if (sum_other != "1"):
            print("Test 19 Failed - sum_other not expected value, should be 1. Was {"+sum_other+"}")
            return 0
        
        [cookies,act,flag] = checked_logout(19,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 19 Passed")
        return 1
    except:
        print("Test 19 Failed - Exception Caused.")
        return 0

In [24]:
# Test 20 - summary present with some added traffic including undo
def test20():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(20,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        # ignoring any errors here
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",2,"car")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle")
        [cookies,act] = do_undo(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",2,"motorbike")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle") 
        [cookies,act] = do_add(cookies,"road",1,"car")
        [cookies,act] = do_add(cookies,"avenue",2,"taxi")
        [cookies,act] = do_add(cookies,"nowhere",3,"taxi")
        [cookies,act] = do_undo(cookies,"main",2,"van")
        [cookies,act] = do_add(cookies,"nowhere",4,"truck")
        [cookies,act] = do_add(cookies,"avenue",2,"bus")
        [cookies,act] = do_add(cookies,"lane",3,"other")
        [cookies,act] = do_add(cookies,"road road",2,"bicycle")
        [cookies,act] = do_add(cookies,"lane",2,"other")
        [cookies,act] = do_summary(cookies)
        
        sum_car = find_refill(act,'sum_car')
        if (sum_car == None):
            print("Test 20 Failed - sum_car not present")
            return 0
            
        if (sum_car != "2"):
            print("Test 20 Failed - sum_car not expected value, should be 2. Was {"+sum_car+"}")
            return 0
        
        sum_taxi = find_refill(act,'sum_taxi')
        if (sum_taxi == None):
            print("Test 20 Failed - sum_taxi not present")
            return 0
            
        if (sum_taxi != "2"):
            print("Test 20 Failed - sum_taxi not expected value, should be 2. Was {"+sum_taxi+"}")
            return 0
        
        sum_bicycle = find_refill(act,'sum_bicycle')
        if (sum_bicycle == None):
            print("Test 20 Failed - sum_bicycle not present")
            return 0
            
        if (sum_bicycle != "3"):
            print("Test 20 Failed - sum_bicycle not expected value, should be 3. Was {"+sum_bicycle+"}")
            return 0
        
        sum_van = find_refill(act,'sum_van')
        if (sum_van == None):
            print("Test 20 Failed - sum_van not present")
            return 0
            
        if (sum_van != "0"):
            print("Test 20 Failed - sum_van not expected value, should be 0. Was {"+sum_van+"}")
            return 0
        
        sum_truck = find_refill(act,'sum_truck')
        if (sum_truck == None):
            print("Test 20 Failed - sum_truck not present")
            return 0
            
        if (sum_truck != "1"):
            print("Test 20 Failed - sum_truck not expected value, should be 1. Was {"+sum_truck+"}")
            return 0
        
        sum_motorbike = find_refill(act,'sum_motorbike')
        if (sum_motorbike == None):
            print("Test 20 Failed - sum_motorbike not present")
            return 0
            
        if (sum_motorbike != "1"):
            print("Test 20 Failed - sum_motorbike not expected value, should be 1. Was {"+sum_motorbike+"}")
            return 0
        
        sum_bus = find_refill(act,'sum_bus')
        if (sum_car == None):
            print("Test 20 Failed - sum_bus not present")
            return 0
            
        if (sum_bus != "3"):
            print("Test 20 Failed - sum_bus not expected value, should be 3. Was {"+sum_bus+"}")
            return 0
        
        sum_other = find_refill(act,'sum_other')
        if (sum_other == None):
            print("Test 20 Failed - sum_other not present")
            return 0
            
        if (sum_other != "2"):
            print("Test 20 Failed - sum_other not expected value, should be 2. Was {"+sum_other+"}")
            return 0
        
        [cookies,act,flag] = checked_logout(19,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 20 Passed")
        return 1
    except:
        print("Test 20 Failed - Exception Caused.")
        return 0

In [25]:
# Test 21 - two users different cookies
def test21():
    try:
        cookies1 = ['','']
        [cookies1,act1,flag1] = checked_login(21,cookies1,"test1","password1")
        if flag1!=True:
            return 0
        
        cookies2 = ['','']
        [cookies2,act2,flag2] = checked_login(21,cookies2,"test2","password2")
        if flag1!=True:
            return 0
        
        if (cookies1[0] == cookies2[0]):
            print("Test 21 - Cookie username matches for dual login.")
            return 0
        
        if (cookies1[1] == cookies2[1]):
            print("Test 21 - Cookie magic matches for dual login.")
            return 0    

        [cookies1,act1,flag1] = checked_logout(21,cookies1)        
        where1 = find_redirect(act1) 
        if flag1!=True:
            return 0
        
        [cookies2,act2,flag2] = checked_logout(21,cookies2)        
        where1 = find_redirect(act1) 
        if flag1!=True:
            return 0

        print("Test 21 Passed")
        return 1
    except:
        print("Test 21 Failed - Exception Caused.")
        return 0

In [26]:
# Test 22 - same user different cookies on second login
def test22():
    try:
        cookies1 = ['','']
        [cookies1,act1,flag1] = checked_login(22,cookies1,"test1","password1")
        if flag1!=True:
            return 0
        
        [cookiesx,act1,flag1] = checked_logout(22,cookies1)        
        where1 = find_redirect(act1) 
        if flag1!=True:
            return 0
        
        cookies2 = ['','']
        [cookies2,act2,flag2] = checked_login(22,cookies2,"test1","password1")
        if flag1!=True:
            return 0
        
        [cookiesx,act2,flag2] = checked_logout(22,cookies2)        
        where1 = find_redirect(act1) 
        if flag1!=True:
            return 0
        
        if (cookies1[1] == cookies2[1]):
            print("Test 22 - Cookie magic reused for second login.")
            return 0    

        print("Test 22 Passed")
        return 1
    except:
        print("Test 22 Failed - Exception Caused.")
        return 0

In [27]:
# Test 23 - two users sessions should not overlap totals
def test23():
    try:
        cookies1 = ['','']
        [cookies1,act1,flag1] = checked_login(23,cookies1,"test1","password1")
        if flag1!=True:
            return 0
        
        cookies2 = ['','']
        [cookies2,act2,flag2] = checked_login(23,cookies2,"test2","password2")
        if flag2!=True:
            return 0
        
        [cookies1,act] = do_add(cookies1,"road",1,"car")
        
        where = find_redirect(act)
    
        if (where != None):
            print("Test 23 Failed - Unexpected redirect.")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 23 Failed - No acknowledge message present")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 23 Failed - No total message present")
            return 0
        
        if (total != "1"):
            print("Test 23 Failed - Total not expected value 1 was "+total)
            return 0

        [cookies2,act] = do_add(cookies2,"road",1,"bus")
            
        where = find_redirect(act)
    
        if (where != None):
            print("Test 23 Failed - Unexpected redirect.")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 23 Failed - No acknowledge message present")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 23 Failed - No total message present")
            return 0
        
        if (total != "1"):
            print("Test 23 Failed - Total not expected value 1 was "+total)
            return 0
        
        [cookies2,act] = do_add(cookies2,"road",1,"bicycle")
            
        where = find_redirect(act)
    
        if (where != None):
            print("Test 23 Failed - Unexpected redirect.")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 23 Failed - No acknowledge message present")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 23 Failed - No total message present")
            return 0
        
        if (total != "2"):
            print("Test 23 Failed - Total not expected value 2 was "+total)
            return 0

        [cookies1,act1,flag1] = checked_logout(23,cookies1)        
        where1 = find_redirect(act1) 
        if flag1!=True:
            return 0
        
        [cookies2,act2,flag2] = checked_logout(23,cookies2)        
        where1 = find_redirect(act1) 
        if flag1!=True:
            return 0

        print("Test 23 Passed")
        return 1
    except:
        print("Test 23 Failed - Exception Caused.")
        return 0

In [28]:
# Test 24 - Simple add with malformed occupancy
def test24():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(24,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies,act] = do_add(cookies,"road","two","car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 24 Failed - Unexpected redirect.(2)")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 24 Failed - No message present")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 24 Failed - No total present")
            return 0
        
        if (total != "0"):
            print("Test 24 Failed - Total not expected value")
            return 0
        
        [cookies,act,flag] = checked_logout(4,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 24 Passed")
        return 1
    except:
        print("Test 24 Failed - Exception Caused.")
        return 0

In [29]:
# Test 25 - Fetch the hours summary csv, check all users included
def test25():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(25,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies,content] = do_file_fetch(cookies,"/statistics/hours.csv")
        
        csv_content = csv.reader(content.splitlines())
        
        if (len(list(csv_content)) != 11):
            print("Test 25 Failed - Unexpected length of csv file.")
            return 0
        
              
        [cookies,act,flag] = checked_logout(25,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 25 Passed")
        return 1
    except:
        print("Test 25 Failed - Exception Caused.")
        return 0

In [30]:
# Test 26 - Fetch the hours summary csv, check 1st user results
def test26():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(26,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies,content] = do_file_fetch(cookies,"/statistics/hours.csv")
        
        found = 0
        for row in content.splitlines():
            if row.startswith("test1,1.0,3.0,3.0"):
                found = 1
            if row.startswith("'test1',1.0,3.0,3.0"): # accept the variants
                found = 1
            if row.startswith('"test1",1.0,3.0,3.0'):
                found = 1

        if found == 0:
            print("Test 26 Failed to calculate correct value for user test1")
            return 0
            
        [cookies,act,flag] = checked_logout(26,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 26 Passed")
        return 1
    except:
        print("Test 26 Failed - Exception Caused.")
        return 0

In [31]:
# Test 27 - Fetch the hours summary csv, check 2nd and 3rd user
def test27():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(27,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies,content] = do_file_fetch(cookies,"/statistics/hours.csv")
        
        found = 0
        for row in content.splitlines():
            if row.startswith("test2,0.0,2.0,5.0"):
                found = 1
            if row.startswith("'test2',0.0,2.0,5.0"):
                found = 1
            if row.startswith('"test2",0.0,2.0,5.0'):
                found = 1

        if found == 0:
            print("Test 27 Failed to calculate correct value for user test1 (user3 unchecked)")
            return 0
        
        found = 0
        for row in content.splitlines():
            if row.startswith("test3,1.0,1.0,1.0"):
                found = 1
            if row.startswith("'test3',1.0,1.0,1.0"):
                found = 1
            if row.startswith('"test3",1.0,1.0,1.0'):
                found = 1

        if found == 0:
            print("Test 27 Failed to calculate correct value for user user3 (test2 ok)")
            return 0
              
        [cookies,act,flag] = checked_logout(27,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 27 Passed")
        return 1
    except:
        print("Test 27 Failed - Exception Caused.")
        return 0

In [32]:
# Test 28 - Fetch the traffic summary, correct number of rows
def test28():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(28,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies2,content] = do_file_fetch(cookies,"/statistics/traffic.csv")
        
        csv_content = csv.reader(content.splitlines())
        
        if (len(list(csv_content)) != 5):
            print("Test 28 Failed - Unexpected length of csv file.")
            return 0
        
              
        [cookies,act,flag] = checked_logout(28,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 28 Passed")
        return 1
    except:
        print("Test 28 Failed - Exception Caused.")
        return 0

In [33]:
# Test 29 - Fetch the traffic summary, correct add values
def test29():
    match = [['main road','car','1','2','0','1'],
                   ['main road','bicycle','0','1','0','1'],
                   ['other road','car','1','0','0','0'],
                   ['other road','taxi','0','0','1','0']]

    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(29,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies2,content] = do_file_fetch(cookies,"/statistics/traffic.csv")
        found = [0,0,0,0]
        csv_content = csv.reader(content.splitlines())
        for row in list(csv_content):
            for i in range(4):
                if (match[i][0] in row[0]) and (match[i][1] in row[1]) and (match[i][2] == row[2]) and (match[i][3] == row[3]) and (match[i][4] == row[4]) and (match[i][5] == row[5]):
                    found[i] += 1
                    
        for f in found:
            if f != 1:
                print("Test 29: Expected row not found exactly once.")
                return 0
        
        [cookies,act,flag] = checked_logout(29,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 29 Passed")
        return 1
    except:
        print("Test 29 Failed - Exception Caused.")
        return 0

In [34]:
# Test 30 - Fetch the traffic summary, add, undo, traffic from previous days
def test30():
    match2 = [['disc drive','other','5','0','0','0'],
               ['main road','bicycle','3','0','0','0'],
               ['main road','bus','1','0','0','3'],
               ['other road','car','0','1','0','0']]
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(30,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies2,content] = do_file_fetch(cookies,"/statistics/traffic.csv")
        
        found = [0,0,0,0]
        
        csv_content = csv.reader(content.splitlines())
        
        listed = list(csv_content)
        
        crow = 0
        for row in listed:
            crow += 1
            for i in range(4):
                if (match2[i][0] in row[0]) and (match2[i][1] in row[1]) and (match2[i][2] == row[2]) and (match2[i][3] == row[3]) and (match2[i][4] == row[4]) and (match2[i][5] == row[5]):
                    found[i] += 1
                    
        for f in found:
            if f != 1:
                print("Test 30: Expected row not found exactly once.")
                return 0
            
        if crow != 5:
            print("Test 30 Failed - Unexpected length of csv file.")
            return 0
        
        [cookies,act,flag] = checked_logout(29,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 30 Passed")
        return 1
    except:
        print("Test 30 Failed - Exception Caused.")
        return 0

In [35]:
# Test 31 - Missing location in add
def test31():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(31,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
    
        [cookies_x,act] = do_add(cookies,"",1,"car")
        where = find_redirect(act)
    
        if (where != None):
            print("Test 31 Failed - Unexpected redirect.(2)")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 31 Failed - No acknowledge message present(3)")
            return 0
        
        total = find_refill(act,'total')
    
        if (total != None):
            if (total != "0"):
                print("Test 31 Failed - Total not expected value, should be zero.")
                return 0
        
        [cookies,act,flag] = checked_logout(31,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 31 Passed")
        return 1
    except:
        print("Test 31 Failed - Exception Caused.")
        return 0

In [36]:
# Test 32 - Vehicle type missing in add
def test32():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(32,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
    
        [cookies_x,act] = do_add(cookies,"road",1,"")
        where = find_redirect(act)
    
        if (where != None):
            print("Test 32 Failed - Unexpected redirect.(2)")
            return 0
        
        message = find_refill(act,'message')
    
        if (message == None):
            print("Test 32 Failed - No acknowledge message present(3)")
            return 0
        
        total = find_refill(act,'total')
    
        if (total != None):
            if (total != "0"):
                print("Test 32 Failed - Total not expected value, should be zero.")
                return 0
        
        [cookies,act,flag] = checked_logout(31,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 32 Passed")
        return 1
    except:
        print("Test 32 Failed - Exception Caused.")
        return 0

In [37]:
# Test 33 - Check undo only within session.
def test33():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(33,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies_x,act] = do_add(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 33 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 33 Failed - No total message present")
            return 0
        
        [cookies,act,flag] = checked_logout(33,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        cookies = ['','']
        [cookies,act,flag] = checked_login(33,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
        
        [cookies_x,act] = do_add(cookies,"road",1,"car")
        
        [cookies_x,act] = do_undo(cookies,"road",1,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 33 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
    
        if (total == None):
            print("Test 33 Failed - No total message present")
            return 0
        
            if (total != "0"):
                print("Test 33 Failed - Total not expected value, should be zero.")
                return 0
        
        [cookies,act,flag] = checked_logout(5,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0 
        
        print("Test 33 Passed")
        return 1
    except:
        print("Test 33 Failed - Exception Caused.")
        return 0

In [38]:
# Test 34 - Undo with wrong location
def test34():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(34,cookies,"test1","password1")
        where = find_redirect(act)
        if flag!=True:
            return 0
            
        [cookies_x,act] = do_add(cookies,"main road",2,"car")
      
        where = find_redirect(act)
    
        if (where != None):
            print("Test 34 Failed - Unexpected redirect.(2)")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):
            if (total != "1"):
                print("Test 34 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
        
        [cookies_x,act] = do_undo(cookies,"other road",2,"car")
      
        
        message = find_refill(act,'message')
        
        if (message == None):
            print("Test 34 Failed - No error message present")
            return 0
        
        total = find_refill(act,'total')
        
        if (total != None):   
            if (total != "1"):
                print("Test 34 Failed - Total not expected value, should be 1. Was {"+total+"}")
                return 0
            
        where = find_redirect(act)
        
        if (where != None):
            print("Test 34 Failed - Unexpected redirect.(2)")
            return 0
        
        [cookies,act,flag] = checked_logout(34,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0

        print("Test 34 Passed")
        return 1
    except:
        print("Test 34 Failed - Exception Caused.")
        return 0

In [39]:
# Test 35 - Try a different set of users, successful login
def test35():
    try:
        cookies = ['','']
        [cookies,act,flag] = checked_login(35,cookies,"usera","ardvark")
        where = find_redirect(act)
        if flag!=True:
            return 0

        [cookies,act,flag] = checked_logout(35,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 35 Passed")
        return 1
    except:
        print("Test 35 Failed - Exception Caused.")
        return 0

In [40]:
# Test 36 - Try a different set of users, bad successful login
def test36():
    try:
        cookies = ['','']
        [cookies,act2] = do_login(cookies,"test1","password1") # for the database given, this should not work.
        where = find_redirect(act2)
    
        if (where != None):
            print("Test 36 Failed - Unexpected redirect.")
            return 0
        
        message = find_refill(act2,'message')
    
        if (message == None):
            print("Test 36 Failed - No failure message present")
            return 0
        
        [cookies,act,flag] = checked_logout(2,cookies)        
        where = find_redirect(act) 
        if flag!=True:
            return 0
        
        print("Test 36 Passed")
        return 1
    except:
        print("Test 36 Failed - Exception Caused.")
        return 0 

In [41]:
tests = [('Test 1 Simple login test.', test1, 'db/clean.db'),
         ('Test 2 Check bad password rejected.', test2, 'db/clean.db'),
         ('Test 3 Login injection attack', test3, 'db/clean.db'),
         ('Test 4 Simple add traffic.', test4, 'db/clean.db'),
         ('Test 5 Multiple adds.', test5, 'db/clean.db'),
         ('Test 6 Check total is per session.', test6, 'db/clean.db'),
         ('Test 7 Per session total, mixed traffic.', test7, 'db/clean.db'),
         ('Test 8 Reject unknown vehicle type.', test8, 'db/clean.db'),
         ('Test 9 Second unknown vehicle type test.', test9, 'db/clean.db'),
         ('Test 10 Check simple undo.', test10, 'db/clean.db'),
         ('Test 11 Check undo exceeding add is rejected.', test11, 'db/clean.db'),
         ('Test 12 Check undo is matched properly', test12, 'db/clean.db'),
         ('Test 13 Everyone wins', test13, 'db/clean.db'),
         ('Test 14 Check out of range occupancy rejected for add.', test14, 'db/clean.db'),
         ('Test 15 Check out of range occupancy rejected for undo.', test15, 'db/clean.db'),
         ('Test 16 Check for injection to undo location.', test16, 'db/clean.db'),
         ('Test 17 Check for injection to undo type.', test17, 'db/clean.db'),
         ('Test 18 Check per session summary all zero on start.', test18, 'db/clean.db'),
         ('Test 19 Check summary after some traffic added.', test19, 'db/clean.db'),
         ('Test 20 Check summary after some traffic added, with undo', test20, 'db/clean.db'),
         ('Test 21 Check different users get different cookies.', test21, 'db/clean.db'),
         ('Test 22 Check same user gets different magic per session.', test22, 'db/clean.db'),
         ('Test 23 Check two users sessions do not overlap totals.', test23, 'db/clean.db'),
         ('Test 24 Check simple add with occupancy non-numeric value.', test24, 'db/clean.db'),
         ('Test 25 Fetch hours csv, all users have hours, even if zero.', test25, 'db/hoursa.db'),
         ('Test 26 Fetch hours csv, hours correctly calculated, 1st user.', test26, 'db/hoursa.db'),
         ('Test 27 Fetch hours csv, hours correctly calculated, 2nd+3rd user.', test27, 'db/hoursa.db'),
         ('Test 28 Fetch count summary csv, correct number of rows', test28, 'db/traffa.db'),
         ('Test 29 Fetch count summary csv, just adds in one day', test29, 'db/traffa.db'),
         ('Test 30 Fetch count summary csv, adds, undo and some data in previous day', test30, 'db/traffb.db'),
         ('Test 31 Check if location missing in add.', test31, 'db/clean.db'),
         ('Test 32 Check if vehicle type missing in add', test32, 'db/clean.db'),
         ('Test 33 Check undo only within session.', test33, 'db/clean.db'),
         ('Test 34 Undo with wrong location.', test34, 'db/clean.db'),
         ('Test 35 Check for hardcoded users. Accept users from database.', test35, 'db/otherusers.db'),
         ('Test 36 Check for hardcoded users. Dont accept missing users.', test36, 'db/otherusers.db')
        ]

def run_test_suite():

    mark_total = 0
        
    for test in tests:
        print("Running "+test[0])
        try:
            shutil.copy(test[2],'traffic.db')
            sp = subprocess.Popen('C:\\Users\\LENOVO\\AppData\\Local\\Programs\\Python\\Python39\\python.exe server.py '+server_port,stdout=subprocess.DEVNULL)
            mark_total += test[1]()
            print("Test Finished")
            sp.terminate()
        except:
            try:
                sp.terminate()
            except:
                pass
            print("Test Process Generated Exception")
        
    print("Marks = {:d}/36 for regression tests.".format(mark_total))

In [42]:
run_test_suite()

Running Test 1 Simple login test.
Test 1 Passed
Test Finished
Running Test 2 Check bad password rejected.
Test 2 Passed
Test Finished
Running Test 3 Login injection attack
Test 3 Passed
Test Finished
Running Test 4 Simple add traffic.
Test 4 Passed
Test Finished
Running Test 5 Multiple adds.
Test 5 Passed
Test Finished
Running Test 6 Check total is per session.
Test 6 Passed
Test Finished
Running Test 7 Per session total, mixed traffic.
Test 7 Passed
Test Finished
Running Test 8 Reject unknown vehicle type.
Test 8 Passed
Test Finished
Running Test 9 Second unknown vehicle type test.
Test 9 Passed
Test Finished
Running Test 10 Check simple undo.
Test 10 Passed
Test Finished
Running Test 11 Check undo exceeding add is rejected.
Test 11 Passed
Test Finished
Running Test 12 Check undo is matched properly
Test 12 Passed
Test Finished
Running Test 13 Everyone wins
Test 13 Passed
Test Finished
Running Test 14 Check out of range occupancy rejected for add.
Test 14 Passed
Test Finished
Running 